First things first.  Load the package.

In [ ]:
Sys.setenv(R_CONFIG_ACTIVE = "jupyter")
suppressMessages({
  devtools::load_all()
  data(pcd)
})

logger::log_threshold(logger::INFO, "global",index = 2)

## publishing context

This article addresses publishing context.  The workflow scenario is authoring documentation in a Jupyter notebook.  The requirement is to support rmarkdown html, rmarkdown pdf, and pkgdown article output.  Details of these document pipelines are in the appendix.

### jupyter yaml

It's handy to have a copy & paste YAML header for jupyter documents:

```yaml
---
title: "authoring"

all_doc: &shared
  pandoc_args: 
    - !with_env "--resource-path={R_HERE_HERE}/vignettes"

output:
  html_document:
    <<: *shared
    theme: cerulean
    highlight: espresso
  pdf_document:
    <<: *shared
---
```

## with_pander

`pander` converts to markdown.  `with_pander` adds logging and context dependent output.

In [ ]:
# convert to markdown, with a message
pander({
  message("hello, stderr")
  pcd
})

In [ ]:
# publishing contextualized upgrade!
with_pander({
  message("hello, stderr.  hello, log.")
  pcd
})

## sure_plot

`with_pander` can be used with plots.  It does this by generates an image file and injecting a markdown link.  By default the image format is PNG, which is fine for HTML, but isn't ideal for PDF output.

In [ ]:
# generate a PNG file
with_pander({
  plot(pcd$x, pcd$y)
})

A PDF screen grab illustrates the image scaling artifacts.

![](figure/pdf-raster.png)

`sure_plot` addresses this concern via publishing context.

In [ ]:
# Produce a context aware plot
sure_plot(
  {
    plot(pcd$x, pcd$y)
  },
  height = 5,
  width = 5,
  res = 96,
  par_opts = list(bg = "azure")
)

And the corresponding screen grab from the PDF.

![](figure/pdf-vector.png)

## appendix

### publishing formats

This snippet builds all three output formats:

```r
article_name <- "authoring"
ipynb_in <- here::here("vignettes", glue("{article_name}.ipynb"))

rmd_out <- fs::path_ext_set(ipynb_in, ".Rmd")
rmd_out <- pub_ipynb_to_rmd(ipynb_in, rmd_out)

# build html and pdf as per the jupyter YAML header
knitr_set_config()
rmarkdown::render(
  input = rmd_out,
  output_format = "all"
)

# build the pkgdown article
pkgdown::build_article("authoring")
```


### incremental results

We can also build just an Rmarkdown file.  This can be processed with knitr to obtain a regular markdown file, the latter being an intermediary step in the above.


```r
article_base <- here::here("vignettes/authoring")
ipynb <- fs::path_ext_set(article_base, ".ipynb")
rmd <- fs::path_ext_set(article_base, ".Rmd")
md <- fs::path_ext_set(article_base, ".md")

pub_ipynb_to_rmd(ipynb, rmd) %>%
  knitr::knit(md)
```


However, we advice against this because, at the markdown stage, the final format is not necessarily determined.  We offer the following (`knitr_set_config`) to adjust the output format of `sure_plot` to meet downstream needs:

```r
# set the knitr configuration to enable 'jpeg' output from sure_plot
knitr_set_config(dev = "jpeg")
knitr::knit(rmd, md)
```